In [1]:
# import particles 
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Input
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
# def next_angle(q, u):
#     # q_init: array (2,) containing the inital angles
#     # u:      array (2,) containing the change rate
#     return q + u

def current_position(q, l):
    # q: array (2,) containing the two angles
    # l: array (2,) containing the limb lengths 
    x = l * np.cos(q[0]) + l * np.cos(q[0] + q[1])
    y = l * np.sin(q[0]) + l * np.sin(q[0] + q[1])
    return np.array([x, y])

# Data Simulation (comprehension 0)

In [3]:
n_tasks = 1000
n_episode = 150
n_timesteps = 10
dof = 2

angles = list()
positions = list()
# generate limb length for each task
lens = np.random.normal(loc=1, scale=np.sqrt(0.3), size=n_tasks)
# lens = np.random.normal(loc=1, scale=np.sqrt(0.3), size=(n_tasks, 2))
# for each task, generate {n_episode} episodes, each contains {n_timesteps} timesteps
for i in range(n_tasks):
  limb_length = lens[i]
  for _ in range(n_episode):
    q = np.random.uniform(low= -2*np.pi, high= 2*np.pi, size=dof)# observation, the angle, uniform [-pi, pi]
    angles.append(q)
    u = np.random.randn(2,n_timesteps) # action, the change rate of angles
    q += u.sum(axis=1)
    position = current_position(q, limb_length)
    positions.append(position)

angles = np.stack(angles, axis=0)
positions = np.stack(positions, axis=0)

assert angles.shape == positions.shape # their shape should be both (n_tasks*n_episode*n_timesteps, 2)

In [4]:
positions.shape, angles.shape

((150000, 2), (150000, 2))

# Split data

In [5]:
x_train, x_test, y_train, y_test = train_test_split(angles, positions, test_size=0.1, random_state=666)

# Build model

In [20]:
input_length = 2
latent_dim = 512
output_length = 2

inputs = Input(shape=(input_length,))
x = Dense(units=latent_dim, activation='relu')(inputs)
# x = Dropout(rate=0.5)(x)
x = Dense(units=latent_dim, activation='relu')(x)
x = Dropout(rate=0.5)(x)
outputs = Dense(units=output_length)(x)
model = Model(inputs=inputs, outputs=outputs, name="Two-Link-Arm") # no space in the name!

In [21]:
model.summary()

Model: "Two-Link-Arm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_11 (Dense)            (None, 512)               1536      
                                                                 
 dense_12 (Dense)            (None, 512)               262656    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 2)                 1026      
                                                                 
Total params: 265,218
Trainable params: 265,218
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.losses.MeanAbsoluteError()])
            
model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_test, y_test))

Epoch 1/100
1055/1055 [==============================] - 4s 3ms/step - loss: 1.1757 - mean_absolute_error: 0.7905 - val_loss: 1.0962 - val_mean_absolute_error: 0.7586
Epoch 2/100
1055/1055 [==============================] - 3s 3ms/step - loss: 1.0427 - mean_absolute_error: 0.7465 - val_loss: 0.9596 - val_mean_absolute_error: 0.7156
Epoch 3/100
1055/1055 [==============================] - 3s 3ms/step - loss: 0.9223 - mean_absolute_error: 0.7059 - val_loss: 0.8602 - val_mean_absolute_error: 0.6762
Epoch 4/100
1055/1055 [==============================] - 3s 3ms/step - loss: 0.8319 - mean_absolute_error: 0.6687 - val_loss: 0.7624 - val_mean_absolute_error: 0.6256
Epoch 5/100
1055/1055 [==============================] - 3s 3ms/step - loss: 0.7652 - mean_absolute_error: 0.6395 - val_loss: 0.6932 - val_mean_absolute_error: 0.5905
Epoch 6/100
1055/1055 [==============================] - 3s 3ms/step - loss: 0.7191 - mean_absolute_error: 0.6167 - val_loss: 0.6484 - val_mean_absolute_error: 0.564